In [2]:
'''
In the messy dataset the following things have to be cleaned up: 
- missing values 
- random string values
- swapped columns 
- numeric distortion (numbers have been multiplied, scaled or randomly altered) 
- truncated text
'''
import pandas as pd
import numpy as np
import re

# df = pd.read_csv("../../data/raw/dirty_detailed_listings_data.csv")

# df

In [3]:
# df.isna().sum()

In [ ]:
clean_df = df.copy()

'''
We'll start with missing values and figure out how to handle each column, 
and determing whether it needs to be dropped or not.
'''

# since we have over 96000 records, and only 18 ids missing (Less than 0001%).
# we'll drop the the records with missing ids. ID's should not be fabricated

clean_df = clean_df.dropna(subset=["id"])

# clean_df.isna().sum()

# then we check to ensure all columns are properly converted

clean_df = df.drop(columns=["scrape_id", "source", "estimated_revenue_l365d", "first_review", "last_review"])

def convert_to_numeric(df, wanted_ratio):
    for col in df.columns:
        # first i convert to see how many values were successfully converted
        converted = pd.to_numeric(df[col], errors="coerce")
        
        # here we calculate how many values successfully converted
        convertible_ratio = converted.notna().mean()

        # here we use the wanted to ratio to determine whether you want to convert the column
        if convertible_ratio > wanted_ratio:
            # print(f"Converting '{col}' to numeric ({convertible_ratio*100:.1f}% convertible)")
            df[col] = converted   # Safe to convert
        # else:
            # print(f"NOT converting '{col}' ({convertible_ratio*100:.1f}% convertible)")

    
    return df

def convert_to_string(df, wanted_ratio):
    for col in df.columns:
        # Try numeric conversion temporarily
        converted = pd.to_numeric(df[col], errors="coerce")
        
        # If value is NaN in numeric conversion → it's a non-numeric value
        non_numeric_ratio = converted.isna().mean()
        # print(f"'{col}': {non_numeric_ratio*100:.1f}% string")

        # If mostly non-numeric → convert column to string
        if non_numeric_ratio > wanted_ratio:
            # print(f"  → Converting '{col}' to string")
            df[col] = df[col].astype("string")
        # else:
        #     print(f"  → Keeping '{col}' as is")
        
    return df

# the reason we are converting to Int64 instead of int64, is because some values are missing
# this integer type supports pd.NA
# regular int64 doesnt support this
def convert_percentages_to_int(df, columns):
    for col in columns:
        df[col] = df[col].str.replace("%", "", regex=False)
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

    return df

def convert_floats_to_int(df):
    for col in df.columns:
        if df[col].dtype == "float64":
            missing_val = df[col].dropna()

            if (df[col].dropna() % 1 == 0).all() and len(missing_val) > 0:
                df[col] = df[col].astype("Int64")

    return df 


clean_df = convert_to_numeric(clean_df, 0.5)
clean_df = convert_to_string(clean_df, 0.95)
clean_df = convert_percentages_to_int(clean_df, ["host_response_rate", "host_acceptance_rate"])
clean_df = convert_floats_to_int(clean_df)

# clean_df
# clean_df.dtypes

# non_missing = clean_df["host_id"].dropna()
# bad_host_ids = non_missing[non_missing % 1 != 0]

# bad_host_ids.head(20), bad_host_ids.shape

# i want to convert clearly int columns to integer values. 
# however some of the cells are corrupted and need to be dropped
# before that we need to see which ones have been corrupted

problem_columns = {}
def check_corrupted_float_columns(df):
    
    for col in df.columns:
        if df[col].dtype == "float64":
            non_missing = df[col].dropna()
            bad_values = non_missing[non_missing % 1 != 0]

            if len(bad_values) > 0:
                problem_columns[col] = len(bad_values)

    return problem_columns

clean_df.dtypes


In [14]:
# there are over 79 colums a lot of which contains filler information
# i wanted to look at this from a business perspective so i recorded my insights and figured out what columns could be useful for this
# the rest of the columms, since they are not relevant have been dropped
df = pd.read_csv("../../data/raw/detailed_listings_data.csv")


col_for_insights = [
    'id', 'neighbourhood', 'property_type', 'room_type',
    'price', 'estimated_revenue_l365d',
    'accommodates', 'beds', 'bedrooms', 'bathrooms',
    'review_scores_rating', 'number_of_reviews', 'reviews_per_month',
    'availability_365', 'host_response_rate', 'host_response_time', 'host_since',
    'amenities', 'minimum_nights', 'maximum_nights',
    'latitude', 'longitude', 'neighbourhood_cleansed',
    'host_total_listings_count', 'host_is_superhost',
    'review_scores_cleanliness', 'review_scores_value','host_acceptance_rate'
]

filtered_df = df[col_for_insights]
og_copy = filtered_df.copy()

import pandas as pd
import numpy as np
import re


# Converts the specified columns into a datetime format
def convert_columns_to_datettime(df, columns):
    for col in columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    return df


# Cleans numeric columns by detecting if a high ratio contains $, commas, or numbers
def clean_numeric_columns(df):
    for col in df.columns:
        ser = df[col].astype(str)

        # detect if column appears numeric-like
        numeric_like_ratio = ser.str.contains(r"\d", regex=True).mean()

        if numeric_like_ratio > 0.5:  # column is mostly numeric-like
            df[col] = (
                ser.str.replace("$", "", regex=False)
                   .str.replace(",", "", regex=False)
                   .str.replace(" ", "")
            )
            df[col] = pd.to_numeric(df[col], errors="ignore")

    return df


# Converts percentage columns automatically (if majority of values contain %)
def clean_percentage_columns(df):
    for col in df.columns:
        ser = df[col].astype(str)
        percent_ratio = ser.str.contains("%").mean()

        if percent_ratio > 0.5:  # majority contains %
            df[col] = (
                ser.str.replace("%", "", regex=False)
                   .replace(["nan", "NaN", "None", ""], np.nan)
            )
            df[col] = pd.to_numeric(df[col], errors="coerce") / 100
            
    return df


# Lowercases and trims text-like columns
def clean_text_columns(df):
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).str.strip().str.lower()
    return df


# Converts superhost column to boolean if present
def clean_superhost(df):
    if "host_is_superhost" in df.columns:
        df["host_is_superhost"] = df["host_is_superhost"].map({"t": True, "f": False})
    return df


# Parses amenities into lists if column exists
def clean_amenities(df):
    if "amenities" in df.columns:
        df["amenities"] = (
            df["amenities"]
            .astype(str)
            .str.strip("{}")
            .str.replace('"', "", regex=False)
            .str.split(", ")
        )
    return df


# Standardises null-like text values
def standardise_nulls(df):
    null_like = ["none", "nan", "na", "n/a", "unknown", ""]
    df = df.replace(null_like, pd.NA)
    return df


# Drops duplicate listing IDs
def drop_duplicate_ids(df):
    if "id" in df.columns:
        df = df.drop_duplicates(subset="id")
    return df


# Removes invalid coordinates (0,0)
def clean_coordinates(df):
    if "latitude" in df.columns and "longitude" in df.columns:
        df = df[~((df["latitude"] == 0) & (df["longitude"] == 0))]
    return df


# Removes negative values from all numeric columns
def remove_invalid_numbers(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df = df[df[col].isna() | (df[col] >= 0)]
    return df


# FULL WRAPPER FUNCTION
def clean_airbnb_listings(df):

    df = df.copy()

    # ORDER OF OPERATIONS: simple → complex
    df = standardise_nulls(df)
    df = convert_columns_to_datettime(df, ["host_since"])
    df = clean_text_columns(df)
    df = clean_superhost(df)
    df = clean_amenities(df)
    df = clean_numeric_columns(df)
    df = clean_percentage_columns(df)
    df = drop_duplicate_ids(df)
    df = clean_coordinates(df)
    df = remove_invalid_numbers(df)

    return df

In [10]:
# -----------------------------------------
# TEST CELL — TRACK WHAT CHANGED & WHAT WAS REMOVED
# -----------------------------------------

# 1️⃣ Track original dtypes and shape
original_dtypes = filtered_df.dtypes.copy()
original_shape = filtered_df.shape

# 2️⃣ Track how many rows before each cleaning step
df_test = filtered_df.copy()
rows_before = df_test.shape[0]

# Run each step manually so we can measure row loss
step_rows = {}

# Step 1: Null standardisation
df_step = standardise_nulls(df_test.copy())
step_rows["standardise_nulls"] = rows_before - df_step.shape[0]

# Step 2: Datetime conversion
df_step = convert_columns_to_datettime(df_step.copy(), ["host_since"])
step_rows["convert_columns_to_datetime"] = df_step.shape[0] - df_step.shape[0]  # no row drop

# Step 3: Clean text columns
df_step = clean_text_columns(df_step.copy())
step_rows["clean_text_columns"] = df_step.shape[0] - df_step.shape[0]

# Step 4: Superhost
df_step = clean_superhost(df_step.copy())
step_rows["clean_superhost"] = df_step.shape[0] - df_step.shape[0]

# Step 5: Amenities
df_step = clean_amenities(df_step.copy())
step_rows["clean_amenities"] = df_step.shape[0] - df_step.shape[0]

# Step 6: Numeric cleaning
df_step = clean_numeric_columns(df_step.copy())
step_rows["clean_numeric_columns"] = df_step.shape[0] - df_step.shape[0]

# Step 7: Percentages
df_step = clean_percentage_columns(df_step.copy())
step_rows["clean_percentage_columns"] = df_step.shape[0] - df_step.shape[0]

# Step 8: Drop duplicates
before = df_step.shape[0]
df_step = drop_duplicate_ids(df_step.copy())
step_rows["drop_duplicate_ids"] = before - df_step.shape[0]

# Step 9: Invalid coordinates
before = df_step.shape[0]
df_step = clean_coordinates(df_step.copy())
step_rows["clean_coordinates"] = before - df_step.shape[0]

# Step 10: Invalid negative numbers
before = df_step.shape[0]
df_step = remove_invalid_numbers(df_step.copy())
step_rows["remove_invalid_numbers"] = before - df_step.shape[0]


# 3️⃣ Run the full cleaning pipeline so we can compare
cleaned = clean_airbnb_listings(filtered_df.copy())
new_dtypes = cleaned.dtypes.copy()
new_shape = cleaned.shape

# 4️⃣ Compare dtype changes
dtype_changes = (original_dtypes != new_dtypes)

changed_columns = original_dtypes[dtype_changes]

# 5️⃣ Report summary
print("\n===== 🧪 DATA CLEANING TEST RESULTS =====\n")

print("\n1 Detailed dtype conversions (before → after):\n")

dtype_comparison = pd.DataFrame({
    "before": original_dtypes,
    "after": new_dtypes
})

dtype_changes_table = dtype_comparison[dtype_comparison["before"] != dtype_comparison["after"]]

print(dtype_changes_table)

print("\n2️⃣ Number of columns that changed dtype:", dtype_changes.sum())

print("\n3️⃣ Rows removed by each function:")
for step, removed in step_rows.items():
    print(f"{step}: {removed}")

max_step = max(step_rows, key=step_rows.get)
print(f"\n➡️ Function that removed the most rows: {max_step} ({step_rows[max_step]} rows removed)")

print("\n4️⃣ Original shape:", original_shape)
print("5️⃣ New shape:", new_shape)




/var/folders/yw/8qjcs1g93ws0j0rgq361rm3h0000gn/T/ipykernel_14894/3683481588.py:48: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
/var/folders/yw/8qjcs1g93ws0j0rgq361rm3h0000gn/T/ipykernel_14894/3683481588.py:48: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
/var/folders/yw/8qjcs1g93ws0j0rgq361rm3h0000gn/T/ipykernel_14894/3683481588.py:48: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
/var/folders/yw/8qjcs1g93ws0j0rgq361rm3h0000gn/T/ipykernel_14894/3683481588.py:48: FutureWarning: errors='ignore


===== 🧪 DATA CLEANING TEST RESULTS =====


1 Detailed dtype conversions (before → after):

                            before    after
estimated_revenue_l365d    float64   object
beds                       float64   object
bedrooms                   float64   object
bathrooms                  float64   object
review_scores_rating       float64   object
reviews_per_month          float64   object
host_response_rate          object  float64
host_total_listings_count  float64   object
review_scores_cleanliness  float64   object
review_scores_value        float64   object
host_acceptance_rate        object  float64

2️⃣ Number of columns that changed dtype: 11

3️⃣ Rows removed by each function:
standardise_nulls: 0
convert_columns_to_datetime: 0
clean_text_columns: 0
clean_superhost: 0
clean_amenities: 0
clean_numeric_columns: 0
clean_percentage_columns: 0
drop_duplicate_ids: 0
clean_coordinates: 0
remove_invalid_numbers: 88657

➡️ Function that removed the most rows: remove_invalid_numb

In [11]:
import pandas as pd
import numpy as np
import re

# Converts the specified columns into a datetime format
def convert_columns_to_datetime(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")
    return df

# Cleans numeric-looking columns that were originally object
def clean_numeric_columns(df, original_dtypes):
    for col in df.columns:
        # only touch columns that started as object
        if original_dtypes.get(col) != "object":
            continue

        ser = df[col].astype(str)

        # detect if column appears numeric-like (has digits in most rows)
        numeric_like_ratio = ser.str.contains(r"\d", regex=True).mean()

        if numeric_like_ratio > 0.5:  # column is mostly numeric-like
            ser = (
                ser.str.replace("$", "", regex=False)
                   .str.replace(",", "", regex=False)
                   .str.replace(" ", "")
            )
            df[col] = pd.to_numeric(ser, errors="coerce")

    return df

# Converts percentage columns automatically (if majority of values contain %)
def clean_percentage_columns(df, original_dtypes):
    for col in df.columns:
        # only touch columns that started as object
        if original_dtypes.get(col) != "object":
            continue

        ser = df[col].astype(str)
        percent_ratio = ser.str.contains("%").mean()

        if percent_ratio > 0.5:  # majority contains %
            ser = (
                ser.str.replace("%", "", regex=False)
                   .replace(["nan", "NaN", "None", ""], np.nan)
            )
            df[col] = pd.to_numeric(ser, errors="coerce") / 100
            
    return df

# Lowercases and trims text-like columns (whatever is still object **after** numeric cleaning)
def clean_text_columns(df):
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str).str.strip().str.lower()
    return df

# Converts superhost column to boolean if present
def clean_superhost(df):
    if "host_is_superhost" in df.columns:
        df["host_is_superhost"] = df["host_is_superhost"].map({"t": True, "f": False})
    return df

# Parses amenities into lists if column exists
def clean_amenities(df):
    if "amenities" in df.columns:
        df["amenities"] = (
            df["amenities"]
            .astype(str)
            .str.strip("{}")
            .str.replace('"', "", regex=False)
            .str.split(", ")
        )
    return df

# Standardises null-like text values
def standardise_nulls(df):
    null_like = ["none", "nan", "na", "n/a", "unknown", ""]
    df = df.replace(null_like, pd.NA)
    return df

# Drops duplicate listing IDs
def drop_duplicate_ids(df):
    if "id" in df.columns:
        df = df.drop_duplicates(subset="id")
    return df

# Removes invalid coordinates (0,0)
def clean_coordinates(df):
    if "latitude" in df.columns and "longitude" in df.columns:
        df = df[~((df["latitude"] == 0) & (df["longitude"] == 0))]
    return df

# Removes negative values from all numeric columns
def remove_invalid_numbers(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df = df[df[col].isna() | (df[col] >= 0)]
    return df

def clean_airbnb_listings(df):
    df = df.copy()

    # keep a record of original dtypes so we know which columns were text
    original_dtypes = df.dtypes.to_dict()

    # ORDER OF OPERATIONS: simple → complex
    df = standardise_nulls(df)
    df = convert_columns_to_datetime(df, ["host_since"])
    df = clean_superhost(df)
    df = clean_amenities(df)

    # numeric & percentage detection based on ORIGINAL object columns
    df = clean_numeric_columns(df, original_dtypes)
    df = clean_percentage_columns(df, original_dtypes)

    # now clean remaining text columns (anything still object)
    df = clean_text_columns(df)

    df = drop_duplicate_ids(df)
    df = clean_coordinates(df)
    df = remove_invalid_numbers(df)

    return df

In [15]:
original_dtypes = og_copy.dtypes.copy()
cleaned = clean_airbnb_listings(filtered_df)

dtype_change_report = pd.DataFrame({
    "before": original_dtypes,
    "after": cleaned.dtypes
})
dtype_change_report = dtype_change_report[dtype_change_report["before"] != dtype_change_report["after"]]

print(dtype_change_report)
print("\nOriginal shape:", filtered_df.shape)
print("New shape:", cleaned.shape)


/var/folders/yw/8qjcs1g93ws0j0rgq361rm3h0000gn/T/ipykernel_14894/289082989.py:49: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
/var/folders/yw/8qjcs1g93ws0j0rgq361rm3h0000gn/T/ipykernel_14894/289082989.py:49: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
/var/folders/yw/8qjcs1g93ws0j0rgq361rm3h0000gn/T/ipykernel_14894/289082989.py:49: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
/var/folders/yw/8qjcs1g93ws0j0rgq361rm3h0000gn/T/ipykernel_14894/289082989.py:49: FutureWarning: errors='ignore' is

                            before    after
estimated_revenue_l365d    float64   object
beds                       float64   object
bedrooms                   float64   object
bathrooms                  float64   object
review_scores_rating       float64   object
reviews_per_month          float64   object
host_response_rate          object  float64
host_total_listings_count  float64   object
review_scores_cleanliness  float64   object
review_scores_value        float64   object
host_acceptance_rate        object  float64

Original shape: (96871, 28)
New shape: (8214, 28)
